In [18]:
from tensorflow.keras.utils import image_dataset_from_directory

images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/cropped/train", seed=1234, subset="both", validation_split=0.2, batch_size=32, labels="inferred", label_mode = "categorical")
images[0]

Found 12000 files belonging to 120 classes.
Using 9600 files for training.
Using 2400 files for validation.
Found 12000 files belonging to 120 classes.
Using 2400 files for validation.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 120), dtype=tf.float32, name=None))>

In [6]:
from tensorflow.keras.applications.vgg16 import VGG16

def load_model():

    model = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

    return model

def set_nontrainable_layers(model):

    model.trainable = False

    return model

model = load_model()
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                              

In [8]:
from tensorflow.keras import layers, models
import tensorflow as tf

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(120, activation='softmax')
    model = tf.keras.Sequential([
      model,
      flattening_layer,
      dense_layer,
      prediction_layer
    ])

    return model

In [9]:
from tensorflow.keras import optimizers

def build_model():
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    model = add_last_layers(base_model)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])
    return model

In [22]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping

aug_images = images[0].map(
  lambda x, y: (preprocess_input(x), y))
aug_images_val = images[1].map(
  lambda x, y: (preprocess_input(x), y))

model = build_model()
model.summary()
es = EarlyStopping(patience=3)
history = model.fit(aug_images,
          epochs=100,
          batch_size=32,
          callbacks=es,
          validation_data=aug_images_val,
          verbose=1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 500)               16384500  
                                                                 
 dense_9 (Dense)             (None, 120)               60120     
                                                                 
Total params: 31159308 (118.86 MB)
Trainable params: 16444620 (62.73 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
Epoch 1/100
300/300 [==============================] - 70s 228ms/step - loss: 6.3054 - accuracy: 0.1003 - val_loss: 3.8339 - val_accuracy